In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

TensorFlow 1.x selected.


'1.15.2'

In [ ]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [ ]:
pip install boruta

     |████████████████████████████████| 61kB 3.2MB/s 


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

from tensorflow.keras.datasets import mnist
from tensorflow.python.framework import ops
import warnings, os

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

ops.reset_default_graph()
reset_graph()

'''Preparing==============================================='''
(X_train, y_train), (X_test, y_test) = mnist.load_data()
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X_train = X_train[:1000].reshape(len(X_train[:1000]), 28*28).astype(np.float32)/255.0
y_train = y_train[:1000].astype(np.int32)
X_test = X_test[:250].reshape(len(X_test[:250]), 28*28).astype(np.float32)/255.0
y_test = y_test[:250].astype(np.int32)

'''borute======================================================'''
rf_all_features = RandomForestClassifier(random_state=1, n_estimators=1000, max_depth=5)
rf_all_features.fit(X_train, y_train)

rfc = RandomForestClassifier(random_state=1, n_estimators=1000, max_depth=5)
boruta_selector = BorutaPy(rfc, n_estimators='auto', verbose=2, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train)) 

11493376/11490434 [==============================] - 0s 0us/step
Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	784
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	784
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	784
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	784
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	784
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	784
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	784
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	333
Tentative: 	61
Rejected: 	390
Iteration: 	9 / 100
Confirmed: 	333
Tentative: 	61
Rejected: 	390
Iteration: 	10 / 100
Confirmed: 	333
Tentative: 	61
Rejected: 	390
Iteration: 	11 / 100
Confirmed: 	333
Tentative: 	61
Rejected: 	390
Iteration: 	12 / 100
Confirmed: 	341
Tentative: 	53
Rejected: 	390
Iteration: 	13 / 100
Confirmed: 	341
Tentative: 	53
Rejected: 	390
Iteration: 	14 / 100
Confirmed: 	341
Tentative: 	53
Rejected: 	390
Iteration: 	15 / 100
Confir

BorutaPy(alpha=0.05,
         estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                          class_weight=None, criterion='gini',
                                          max_depth=5, max_features='auto',
                                          max_leaf_nodes=None, max_samples=None,
                                          min_impurity_decrease=0.0,
                                          min_impurity_split=None,
                                          min_samples_leaf=1,
                                          min_samples_split=2,
                                          min_weight_fraction_leaf=0.0,
                                          n_estimators=544, n_jobs=None,
                                          oob_score=False,
                                          random_state=RandomState(MT19937) at 0x7FB192FA46B0,
                                          verbose=0, warm_start=False),
         max_iter=100, n_estimators='a

In [ ]:
X_important_train = boruta_selector.transform(np.array(X_train))
X_important_test = boruta_selector.transform(np.array(X_test))

In [ ]:
X_train = X_important_train.copy()
X_test = X_important_test.copy()

In [ ]:
from sklearn.tree import DecisionTreeClassifier

def DT(X_train, y_train, X_test, y_test, depth=10, split='b', criterion='c'):
  if (criterion == 'e'):
    c='entropy'
  else:
    c='gini'

  if (split == 'b'):
    s='best'
  else:
    s='random'
  
  clf = DecisionTreeClassifier(criterion=c, splitter=s, max_depth=depth)
  clf.fit(X_train, y_train)
  # return clf.predict(X_test)
  print(clf.score(X_train, y_train))
  

In [ ]:
# y_test_new = DT(X_train, y_train, X_test, y_test)
DT(X_train, y_train, X_test, y_test)

0.992


In [ ]:
n_inputs = 349
n_hidden1 = 400
n_hidden2 = 300
n_hidden3 = 200
n_hidden4 = 100
n_outputs = 10

n_epochs = 100
batch_size = 50

n_batches = len(X_train) // batch_size

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')
unit_epoch = tf.placeholder(dtype=tf.float32, shape=None)

def learning_shedule(epoch):
    return 1 / ((epoch+1) * 100)

# def batch_forward(X, eps=1e-5):
#     beta = sample_mean = X.mean(axis=0)
#     sample_var = X.var(axis=0)
#     gamma = np.sqrt(sample_var + eps)

#     X_norm = (X - sample_mean) / gamma
    
#     z_out = gamma*X_norm + beta
#     return z_out

def batch_forward(X, eps=1e-5):
    X = tf.cast(X, dtype=tf.float32)
    sample_mean, sample_variance = tf.math.reduce_mean(X), tf.math.reduce_variance(X)
    mean, variance = tf.math.reduce_mean(tf.cast(X_train, dtype=tf.float32)), tf.math.reduce_variance(tf.cast(y_train, dtype=tf.float32))
    
    gamma = tf.sqrt(variance)
    beta = mean
    
    X_norm = (X - sample_mean) / tf.sqrt(sample_variance + eps)
    return (gamma * X_norm) + beta 
    

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index) 
    indices = np.random.randint(len(X_train), size=batch_size) 
    X_batch = X_train[indices]
    y_batch = y_train[indices]
    return X_batch, y_batch


def neuron_layer(X, n_neurals, name, activation=None):
    with tf.name_scope(name):
        '''weights initialization======================================================================'''
        
        init = np.random.randn(int(X.get_shape()[1]), n_neurals) * 4 * np.sqrt(2 / (int(X.get_shape()[1] + n_neurals)))
        W = tf.Variable(init, name='weights', dtype=tf.float32)
        b = tf.Variable(tf.zeros([n_neurals]), name='biases')
        z = tf.matmul(X, W) + b
        if activation == 'relu':
            return tf.nn.relu(z)
        elif activation == 'elu':
            return tf.keras.activations.elu(z, alpha=0.2)
        elif activation == 'leaky_relu':
            return tf.maximum(0.01 * z, z, name=activation)
        elif activation == 'tanh':
            return tf.math.tanh(z, name=activation)
        else:
            return z
        
with tf.name_scope('dnn'):
    hidden1 = neuron_layer(batch_forward(X), n_hidden1, 'hidden1', activation='tanh')
    hidden2 = neuron_layer(batch_forward(hidden1), n_hidden2, 'hidden2', activation='tanh')
    hidden3 = neuron_layer(batch_forward(hidden2), n_hidden3, 'hidden3', activation='tanh')
    hidden4 = neuron_layer(batch_forward(hidden3), n_hidden4, 'hidden3', activation='tanh')
    logits = neuron_layer(batch_forward(hidden4), n_outputs, 'output')
    
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')

learning_rate = 0.01
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_shedule(unit_epoch))
    training_op = optimizer.minimize(loss)

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)  
    # the max number of index y == argmax()
    # tf.cast(True is 1    False is 0)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    
# '''tensorboard======================================================'''
# now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
# root_logdir = "tf_logs"
# logdir = "{}/run-{}/".format(root_logdir, now)

# mse_summary = tf.summary.scalar('LOSS', loss)
# file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            
            _, _loss = sess.run([training_op, loss], feed_dict={X: X_batch, y: y_batch, unit_epoch: epoch})
            
#             step = epoch * n_batches + iteration
#             summary = tf.Summary(value=[tf.Summary.Value(tag="loss", simple_value=_loss)])
#             file_writer.add_summary(summary, int(step))
        
        train_acc = sess.run(accuracy, feed_dict={X: X_batch, y: y_batch})
        
        test_acc = sess.run(accuracy, feed_dict={X: X_test, y: y_test, unit_epoch: epoch})
        
        if (epoch % 10 == 0):
            print(epoch, "Train accuracy:", train_acc, "Test accuracy:", test_acc)
        
#     save_path = saver.save(sess, "./my_model_final.ckpt")
# file_writer.close()

0 Train accuracy: 0.44 Test accuracy: 0.376
10 Train accuracy: 0.84 Test accuracy: 0.448
20 Train accuracy: 0.9 Test accuracy: 0.524
30 Train accuracy: 0.92 Test accuracy: 0.56
40 Train accuracy: 1.0 Test accuracy: 0.56
50 Train accuracy: 1.0 Test accuracy: 0.532
60 Train accuracy: 1.0 Test accuracy: 0.572
70 Train accuracy: 1.0 Test accuracy: 0.592
80 Train accuracy: 1.0 Test accuracy: 0.628
90 Train accuracy: 0.98 Test accuracy: 0.596
100 Train accuracy: 1.0 Test accuracy: 0.604
110 Train accuracy: 1.0 Test accuracy: 0.596
120 Train accuracy: 1.0 Test accuracy: 0.6
130 Train accuracy: 1.0 Test accuracy: 0.576
140 Train accuracy: 1.0 Test accuracy: 0.596
150 Train accuracy: 1.0 Test accuracy: 0.564
160 Train accuracy: 1.0 Test accuracy: 0.56
170 Train accuracy: 1.0 Test accuracy: 0.576
180 Train accuracy: 1.0 Test accuracy: 0.572
190 Train accuracy: 1.0 Test accuracy: 0.576
200 Train accuracy: 1.0 Test accuracy: 0.58
210 Train accuracy: 1.0 Test accuracy: 0.584
220 Train accuracy: 1.0

In [ ]:
100 Train accuracy: 1.0 Test accuracy: 0.604